In [1]:
import pandas as pd
import datetime as dt
import random

In [2]:
# Get the entire trips database
tripsDF = pd.read_csv('rt_trips_DB_2018.txt', sep=';', usecols=['DAYOFSERVICE', 'TRIPID', 'LINEID', 'ROUTEID', 'DIRECTION'])

In [3]:
tripID_df = pd.read_csv('rt_leavetimes_DB_2018.txt', sep=';', usecols=['TRIPID', 'DAYOFSERVICE', 'PROGRNUMBER', 'STOPPOINTID', 'PLANNEDTIME_ARR', 'ACTUALTIME_ARR', 'ACTUALTIME_DEP'])

In [4]:
tripID_df['STOPPOINTID'].value_counts()

1479    214973
7453    203264
400     201173
1353    199944
1478    199904
         ...  
7566         2
7504         1
7604         1
7627         1
7407         1
Name: STOPPOINTID, Length: 4774, dtype: int64

In [5]:
df = pd.merge(tripsDF, tripID_df, on=['TRIPID', 'DAYOFSERVICE'])

In [6]:
df

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,ACTUALTIME_ARR,ACTUALTIME_DEP
0,07-FEB-18 00:00:00,6253783,68,68_80,1,1,4495,84600,84600,84600
1,07-FEB-18 00:00:00,6253783,68,68_80,1,4,1278,84713,84827,84838
2,07-FEB-18 00:00:00,6253783,68,68_80,1,5,1282,84827,84917,84928
3,07-FEB-18 00:00:00,6253783,68,68_80,1,6,4456,84882,84970,84970
4,07-FEB-18 00:00:00,6253783,68,68_80,1,7,1284,84921,84991,84991
...,...,...,...,...,...,...,...,...,...,...
116949108,14-MAY-18 00:00:00,6765012,27,27_19,1,39,675,43685,43675,43691
116949109,14-MAY-18 00:00:00,6765012,27,27_19,1,49,2314,45033,44869,44888
116949110,14-MAY-18 00:00:00,6765012,27,27_19,1,62,2332,45965,45790,45790
116949111,14-MAY-18 00:00:00,6765012,27,27_19,1,75,4436,46893,46624,46635


In [7]:
df['DAYOFSERVICE'] = pd.to_datetime(df['DAYOFSERVICE'])

In [8]:
df.sort_values(by=['DAYOFSERVICE', 'TRIPID', 'PROGRNUMBER'], inplace=True, ignore_index=True)

### Next stop

In [9]:
df.rename(columns={'STOPPOINTID':'startStop'}, inplace=True)

In [10]:
df['endStop'] = 'N/A'

In [11]:
df.loc[(df['TRIPID']==df['TRIPID'].shift(-1)), 'endStop'] = df['startStop'].shift(-1)

In [12]:
# remove all rows with 'N/A'
dropIndex = list(df[df['endStop']=='N/A'].index)

In [13]:
df.drop(dropIndex, inplace=True)

In [14]:
df.reset_index(drop=True, inplace=True)

In [15]:
df

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PROGRNUMBER,startStop,PLANNEDTIME_ARR,ACTUALTIME_ARR,ACTUALTIME_DEP,endStop
0,2018-01-01,5955277,16,16_20,1,1,7347,30000,30029,30029,3669.0
1,2018-01-01,5955277,16,16_20,1,2,3669,30080,30128,30128,7349.0
2,2018-01-01,5955277,16,16_20,1,3,7349,30138,30182,30182,1631.0
3,2018-01-01,5955277,16,16_20,1,4,1631,30206,30255,30255,1632.0
4,2018-01-01,5955277,16,16_20,1,5,1632,30221,30261,30261,5053.0
...,...,...,...,...,...,...,...,...,...,...,...
114766488,2018-12-31,8592116,7A,7A_87,2,60,494,57676,58324,58339,495.0
114766489,2018-12-31,8592116,7A,7A_87,2,61,495,57796,58424,58424,400.0
114766490,2018-12-31,8592116,7A,7A_87,2,62,400,57893,58490,58515,281.0
114766491,2018-12-31,8592116,7A,7A_87,2,63,281,58218,58841,58910,4.0


### Add JOURNEYTIME first

In [ ]:
# pd.set_option('display.max_rows', 20)
stops = sorted(list(df['startStop'].unique()))

In [ ]:
stopLines = {}
stopNums = {}

for stop in stops:
    print(stop)
    currentStops = sorted(list(df[df['startStop']==stop]['LINEID'].unique()))
    shape = df[df['startStop']==stop].shape[0]
    stopNums[stop] = shape
    stopLines[stop] = currentStops

In [ ]:
# stopLines

In [ ]:
# stopNums

In [ ]:
data = {'stop':list(stopNums.keys()), 'number':list(stopNums.values())}

sdf = pd.DataFrame.from_dict(data)

In [ ]:
sdf.sort_values(by=['number'], ascending=False, ignore_index=True)

In [ ]:
stopNumberList = sorted(list(sdf[sdf['number']>60000]['stop']))

In [ ]:
len(stopNumberList)

In [ ]:
# for stop in sorted(list(sdf[sdf['number']>60000]['stop'])):
#     print(len(stopLines[stop]))

In [ ]:
# this takes 30 mins to run

stopPairDict = {}

for stop in stopNumberList:
    print(stop)
    workingDF = df[df['startStop']==stop]
    for endStop in sorted(list(workingDF['endStop'].unique())):
        workingDFend = workingDF[workingDF['endStop']==endStop]
        if workingDFend.shape[0] > 38000:
            filename = f'FROM_{stop}_TO_{endStop}'
            stopPairDict[filename] = workingDFend

In [ ]:
len(list(stopPairDict.keys()))

### I have dfs for 191 stop-pairs.
- Make this into 1 df

In [ ]:
# stopPairDict.keys()

In [ ]:
frames = [item for item in list(stopPairDict.values())]

In [ ]:
combinedDF = pd.concat(frames)

In [ ]:
combinedDF.reset_index(drop=True, inplace=True)

In [ ]:
combinedDF

In [ ]:
len(list(combinedDF['startStop'].unique()))

In [42]:
countsDF = df.groupby(['startStop', 'endStop']).size().reset_index().rename(columns={0:'count'})

In [45]:
countsDF = countsDF[countsDF['count']>=100000].reset_index(drop=True)

In [ ]:
# countsDF.reset_index(drop=True, inplace=True)

In [52]:
countsDF = countsDF.sort_values(by=['count'], ascending=False).reset_index(drop=True).iloc[0:50]

In [61]:
countsDF

,startStop,endStop,count
0,1478,1479.0,196432
1,1444,1445.0,193150
2,618,619.0,188718
3,617,618.0,188691
4,616,617.0,188616
5,619,675.0,188409
6,615,616.0,184421
7,614,615.0,184405
8,516,4384.0,181962
9,522,7659.0,179068


In [55]:
dfList = []

for i in range(0, countsDF.shape[0]):
    print(i)
    currentDF = df[(df['startStop']==countsDF.loc[i, 'startStop'])&(df['endStop']==countsDF.loc[i, 'endStop'])]
    dfList.append(currentDF)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [56]:
combinedDF = pd.concat(dfList)

In [57]:
combinedDF

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PROGRNUMBER,startStop,PLANNEDTIME_ARR,ACTUALTIME_ARR,ACTUALTIME_DEP,endStop,JOURNEYTIME
1045,2018-01-01,5955344,25B,25B_271,2,33,1478,37184,37186,37210,1479.0,54.0
1119,2018-01-01,5955346,25B,25B_271,2,33,1478,44934,44653,44653,1479.0,21.0
1231,2018-01-01,5955349,25A,25A_270,2,33,1478,71051,70983,70983,1479.0,29.0
1305,2018-01-01,5955351,25A,25A_270,2,33,1478,77408,77399,77399,1479.0,29.0
1379,2018-01-01,5955353,25A,25A_270,2,33,1478,84608,84562,84562,1479.0,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...
112585348,2018-12-31,8591706,27,27_17,2,63,672,60037,60317,60328,4382.0,45.0
112587456,2018-12-31,8591908,15,15_17,2,52,672,32013,32487,32487,4382.0,16.0
112587593,2018-12-31,8591910,15,15_17,2,52,672,46010,45188,45210,4382.0,44.0
112587729,2018-12-31,8591912,15,15_17,2,52,672,58897,58718,58718,4382.0,21.0


In [ ]:
df[(df['startStop']==14)&(df['endStop']==15)].shape[0]

In [58]:
combinedDF.rename(columns={'PLANNEDTIME_ARR':'ARR/DEP_PLAN', 'ACTUALTIME_ARR':'ARR_ACT', 'ACTUALTIME_DEP':'DEP_ACT'},\
                inplace=True)

In [59]:
combinedDF

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PROGRNUMBER,startStop,ARR/DEP_PLAN,ARR_ACT,DEP_ACT,endStop,JOURNEYTIME
1045,2018-01-01,5955344,25B,25B_271,2,33,1478,37184,37186,37210,1479.0,54.0
1119,2018-01-01,5955346,25B,25B_271,2,33,1478,44934,44653,44653,1479.0,21.0
1231,2018-01-01,5955349,25A,25A_270,2,33,1478,71051,70983,70983,1479.0,29.0
1305,2018-01-01,5955351,25A,25A_270,2,33,1478,77408,77399,77399,1479.0,29.0
1379,2018-01-01,5955353,25A,25A_270,2,33,1478,84608,84562,84562,1479.0,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...
112585348,2018-12-31,8591706,27,27_17,2,63,672,60037,60317,60328,4382.0,45.0
112587456,2018-12-31,8591908,15,15_17,2,52,672,32013,32487,32487,4382.0,16.0
112587593,2018-12-31,8591910,15,15_17,2,52,672,46010,45188,45210,4382.0,44.0
112587729,2018-12-31,8591912,15,15_17,2,52,672,58897,58718,58718,4382.0,21.0


In [60]:
combinedDF.to_csv('combinedDF_MODELING_1.csv', index_label=False)

In [ ]:
sorted(list(combinedDF['startStop'].unique()))

In [ ]:
combinedDF[combinedDF['startStop']==619]

### Now I need to get dataframes with all stop-to-stop journeys for each stopnumber in stopNumberList

### JOURNEYTIME feature

In [16]:
df.loc[(df['TRIPID']==df['TRIPID'].shift(-1)), 'JOURNEYTIME'] = (df['ACTUALTIME_ARR'].shift(-1)-df['ACTUALTIME_ARR'])

In [17]:
df

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PROGRNUMBER,startStop,PLANNEDTIME_ARR,ACTUALTIME_ARR,ACTUALTIME_DEP,endStop,JOURNEYTIME
0,2018-01-01,5955277,16,16_20,1,1,7347,30000,30029,30029,3669.0,99.0
1,2018-01-01,5955277,16,16_20,1,2,3669,30080,30128,30128,7349.0,54.0
2,2018-01-01,5955277,16,16_20,1,3,7349,30138,30182,30182,1631.0,73.0
3,2018-01-01,5955277,16,16_20,1,4,1631,30206,30255,30255,1632.0,6.0
4,2018-01-01,5955277,16,16_20,1,5,1632,30221,30261,30261,5053.0,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...
114766488,2018-12-31,8592116,7A,7A_87,2,60,494,57676,58324,58339,495.0,100.0
114766489,2018-12-31,8592116,7A,7A_87,2,61,495,57796,58424,58424,400.0,66.0
114766490,2018-12-31,8592116,7A,7A_87,2,62,400,57893,58490,58515,281.0,351.0
114766491,2018-12-31,8592116,7A,7A_87,2,63,281,58218,58841,58910,4.0,234.0


### Remove null journeys

In [18]:
null = list(sorted(df[df['JOURNEYTIME'].isna()].index))

In [19]:
df.drop(null, inplace=True)

In [20]:
df.reset_index(drop=True, inplace=True)

In [21]:
df

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PROGRNUMBER,startStop,PLANNEDTIME_ARR,ACTUALTIME_ARR,ACTUALTIME_DEP,endStop,JOURNEYTIME
0,2018-01-01,5955277,16,16_20,1,1,7347,30000,30029,30029,3669.0,99.0
1,2018-01-01,5955277,16,16_20,1,2,3669,30080,30128,30128,7349.0,54.0
2,2018-01-01,5955277,16,16_20,1,3,7349,30138,30182,30182,1631.0,73.0
3,2018-01-01,5955277,16,16_20,1,4,1631,30206,30255,30255,1632.0,6.0
4,2018-01-01,5955277,16,16_20,1,5,1632,30221,30261,30261,5053.0,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...
112589629,2018-12-31,8592116,7A,7A_87,2,59,493,57610,58257,58273,494.0,67.0
112589630,2018-12-31,8592116,7A,7A_87,2,60,494,57676,58324,58339,495.0,100.0
112589631,2018-12-31,8592116,7A,7A_87,2,61,495,57796,58424,58424,400.0,66.0
112589632,2018-12-31,8592116,7A,7A_87,2,62,400,57893,58490,58515,281.0,351.0
